#### Name and Surname: Janice Xerri 
#### ID: 0437903L
##### Thesis - Create a personalized and accurate course recommendation system for users in a virtual world based on User Profiling

Importing libraries

In [1]:
import pandas as pd
import os
import numpy as np
import re


Loading the datasets 

In [2]:
columns_to_use = ['Userid', 'Degree 1', 'Degree 1 Specializations', 'Known Languages', 'Key Skills', 'Career Objective']
user_profile_ds = pd.read_csv('Datasets/user_profile_set_21.csv',encoding='unicode_escape', usecols=columns_to_use)
user_rating_ds = pd.read_csv('Datasets/Arranged_User_Ratings.csv',encoding='unicode_escape')
courses_ds = pd.read_csv('Datasets/Master_dataset_Feb22.csv',encoding='unicode_escape')

### User Profile Dataset

In [3]:
print("User Profile Shape: ", user_profile_ds.shape) #stating that the dataset contains 1202 users with 6 columns each
user_profile_ds.head()

User Profile Shape:  (1202, 6)


,Userid,Degree 1,Degree 1 Specializations,Known Languages,Key Skills,Career Objective
0,NaN,NaN,NaN,NaN,NaN,NaN
1,1001.0,B.E.,Computer Science & Engineering,"English, Marathi, Hindi",C; Java; Keras; Flask; Deep Learning; Selenium...,Computer Engineering student with good technic...
2,1002.0,B.E.,Computer Science & Engineering,Hindi English,Java; Neural Networks; AI; Python; Html5; CPP,Interested in working under company offering A...
3,1003.0,B.E.,Computer Science & Engineering,NaN,NaN,NaN
4,1004.0,B.E.,Computer Science & Engineering,"English, Hindi, Marathi, Marwari",XML; C; Java; Data Structures; Python; MongoDB...,Currently a final year student of Computer Eng...


Checking for null values

In [4]:
#check for null values 
user_profile_ds.isnull().sum() #when checking manually the dataset there are 2046 user ids thus, it shows that it also counting empty rows as if they have missing values

Userid                      160
Degree 1                    160
Degree 1 Specializations    160
Known Languages             299
Key Skills                  556
Career Objective            735
dtype: int64

Handling empty rows

In [5]:
# Remove rows where all elements are NaN
user_profile_ds = user_profile_ds.dropna(how='all')

#checking the shape again to see if it matches the expected number of users
print("User Profile Shape after removing empty rows: ", user_profile_ds.shape) #shows that there are 1042 users with 6 columns each

#checking for null values again to confirm
user_profile_ds.isnull().sum() #illustrates that there are some missing values in the known languages, key skills and career objectives data


User Profile Shape after removing empty rows:  (1042, 6)


Userid                        0
Degree 1                      0
Degree 1 Specializations      0
Known Languages             139
Key Skills                  396
Career Objective            575
dtype: int64

Analysing the dataset

In [6]:
#total sum of users 
print("Amount of users: ",len(user_profile_ds['Userid'].unique()))
print("\n")
print(user_profile_ds["Degree 1"].value_counts()) #shows the amount of users that are currently participating within a particular degree
print("\n")
print(user_profile_ds["Degree 1 Specializations"].value_counts()) #shows the amount of users that are currently participating within a particular degree and its specific field



Amount of users:  1042


Degree 1
B.E.    1042
Name: count, dtype: int64


Degree 1 Specializations
Computer Science & Engineering    1042
Name: count, dtype: int64


In [7]:
print(user_profile_ds["Known Languages"].value_counts()) 

Known Languages
Java,CPP                             74
English, Hindi, Marathi              61
C,CPP                                32
C,CPP,Java,Python                    27
English, Marathi, Hindi              23
                                     ..
Java,Linux                            1
Marathi, English, Hindi,Sanskrit      1
Marathi \nEnglish\nHindi              1
English ,Hindi, Marathi               1
English, Hindi, Marathi, Gujarati     1
Name: count, Length: 251, dtype: int64


In [8]:
print(user_profile_ds["Key Skills"].value_counts())

Key Skills
C-Programming; MYSQL; CSS                                                   8
Cpp, Java                                                                   5
C; Java; cpp; MySQL                                                         4
cpp; MySQL                                                                  4
marketing-skill; event-management; presentation-skill                       3
                                                                           ..
JDBC; Java; NetBeans; Excel; HTML; MySQL                                    1
Python; Bootstrap; HTML; Core Java; SQL; CPP; PHP; CSS                      1
cpp; Html,css,java                                                          1
XML; C; Java; Advanced Java; JScript; Mysq; HTML; CPP; Android; PHP; CSS    1
Computer-hardware-assembly; Python; computer-hardware-installation          1
Name: count, Length: 520, dtype: int64


In [9]:
print(user_profile_ds['Career Objective'].value_counts())

Career Objective
To secure a position where I can efficiently contribute my skills and abilities to the growth of the organization and build my professional career.                                                                                          7
To pursue a challenging career and be a part of progressive organization that gives a scope to enhance my knowledge and utilizing my skills towards the growth of the organization.                                                          5
Looking for a challenging role in a reputable organization to utilize my technical, database, and management skills for the growth of the organization as well as to enhance my knowledge about new and emerging trends in the IT sector.    4
Computer Engineering student with good technical skills and problem solving abilities. Areas of interest include Computer Vision, Deep Learning, Machine Learning, and Research.                                                             3
Secure a responsible career

### User Ratings Dataset

In [10]:
print("User Ratings Shape: ", user_rating_ds.shape) #stating that the dataset contains 425 user ratings of 21 courses each
user_rating_ds.head()

User Ratings Shape:  (424, 21)


,Unnamed: 0,1001,1002,1003,1004,1005,1006,1007,1008,1009,...,1011,1012,1013,1014,1015,1016,1017,1018,1019,1020
0,2001,5,3,1,0,2,1,0,0,0,...,3,0,0,4,1,0,2,0,1,3
1,2002,3,5,0,2,1,0,4,5,0,...,0,3,2,0,5,2,0,3,0,0
2,2003,4,1,3,0,2,3,1,4,5,...,1,0,3,2,4,2,3,4,3,4
3,2004,2,4,5,3,3,2,0,4,2,...,4,3,5,2,3,1,3,2,3,0
4,2005,0,4,2,0,1,2,4,3,3,...,1,2,3,0,5,0,0,0,2,1


In [11]:
#checking the names of the columns
print(user_rating_ds.columns)

Index(['Unnamed: 0', '1001', '1002', '1003', '1004', '1005', '1006', '1007',
       '1008', '1009', '1010', '1011', '1012', '1013', '1014', '1015', '1016',
       '1017', '1018', '1019', '1020'],
      dtype='object')


In [12]:
user_rating_ds.head() #for example the 2001 shows a user id and 1001 is a course id

,Unnamed: 0,1001,1002,1003,1004,1005,1006,1007,1008,1009,...,1011,1012,1013,1014,1015,1016,1017,1018,1019,1020
0,2001,5,3,1,0,2,1,0,0,0,...,3,0,0,4,1,0,2,0,1,3
1,2002,3,5,0,2,1,0,4,5,0,...,0,3,2,0,5,2,0,3,0,0
2,2003,4,1,3,0,2,3,1,4,5,...,1,0,3,2,4,2,3,4,3,4
3,2004,2,4,5,3,3,2,0,4,2,...,4,3,5,2,3,1,3,2,3,0
4,2005,0,4,2,0,1,2,4,3,3,...,1,2,3,0,5,0,0,0,2,1


Analysis of the dataset

In [13]:
#calculating the mean 
user_rating_ds.mean() #check the mean of the ratings for each course manually

Unnamed: 0    2212.500000
1001             2.429245
1002             2.775943
1003             2.662736
1004             2.872642
1005             2.594340
1006             2.860849
1007             3.386792
1008             3.009434
1009             2.391509
1010             2.285377
1011             2.179245
1012             2.143868
1013             2.391509
1014             2.356132
1015             2.915094
1016             2.952830
1017             2.584906
1018             1.889151
1019             2.341981
1020             2.870283
dtype: float64

### Courses Dataset

In [14]:
print("Courses Dataset Shape: ", courses_ds.shape) #stating that the dataset contains 10000 courses with 5 columns each
courses_ds.head()

Courses Dataset Shape:  (10000, 5)


,Sr,Degree 1,Degree 1 SpeCializations,Campus,Key Skills
0,1001,B E,Mechanical,MITCOE,CATIA
1,1002,B E,Mechanical,MITCOE,CATIA
2,1003,B E,Mechanical,MITAOE,CATIA
3,1004,B E,Mechanical,MITCOE,CATIA
4,1005,B E,Mechanical,MITCOE,CATIA


Checking for any null values

In [15]:
courses_ds.isnull().sum() #there aren't any null values listed in the dataset

Sr                          0
Degree 1                    0
Degree 1 SpeCializations    0
Campus                      0
Key Skills                  0
dtype: int64

Analysis of the dataset

In [16]:
#finding the value count of Degree 1
print("Number of courses")
print(courses_ds['Degree 1'].value_counts())
print("\n")
print("Number of specialised courses")
print(courses_ds["Degree 1 SpeCializations"].value_counts())
print("\n")
#how many courses occur on different campus
print(courses_ds["Campus"].value_counts())


Number of courses
Degree 1
B E        9115
M TeCh      885
Name: count, dtype: int64


Number of specialised courses
Degree 1 SpeCializations
Mechanical                                    3928
Electronics  Telecommunication Engineering    3280
Computer Science  Engineering                 2208
Civil Engineering                              584
Name: count, dtype: int64


Campus
MITCOE      4935
MITAOE      3323
MIT  WPU     885
MIT,Pune     857
Name: count, dtype: int64


In [17]:
#the counts of unique values in the "Key Skills" column of the courses_ds DataFrame. 
#useful for understanding which key skills are most common or frequent among the courses listed in the dataset.
print(courses_ds["Key Skills"].value_counts())

Key Skills
C, Java, CPP, HTML, CMStool, MYSQL                                                                                                                        1548
AutoCAD, PROE                                                                                                                                             1025
SOLIDWORKS, AUTOCAD, CREO                                                                                                                                  771
CAD,CAM                                                                                                                                                    652
MSCIT, Leadership, AUTOCAD                                                                                                                                 584
AmazonWebServiCes, C CPP, Arduino, MongoDB, Linux, Golang,Microcontrollers, Gobot, InternetofThings, MATLAB, SQL, PHP                                      574
ProE,CATIA                         

### Pre-processing the data as necessary

##### User Profile and Course Dataset

Custom defined functions to pre-process the data

In [18]:
#a function designed to lowercase and change space to _ 
def clean_column_names(df):
    df.columns = df.columns.str.lower().str.replace(' ', '_')
    return df

In [19]:
#taking care of multiple values by splitting them into lists using the split function
def splitting_comma(string_x):
    return (string_x.split(','))

def splitting_semicolon(string_x):
    return (string_x.split(';'))

Applying the custom functions on the datasets accordingly. 

In [20]:
#cleaning column names
user_profile_ds = clean_column_names(user_profile_ds)
user_profile_ds.head()

,userid,degree_1,degree_1_specializations,known_languages,key_skills,career_objective
1,1001.0,B.E.,Computer Science & Engineering,"English, Marathi, Hindi",C; Java; Keras; Flask; Deep Learning; Selenium...,Computer Engineering student with good technic...
2,1002.0,B.E.,Computer Science & Engineering,Hindi English,Java; Neural Networks; AI; Python; Html5; CPP,Interested in working under company offering A...
3,1003.0,B.E.,Computer Science & Engineering,NaN,NaN,NaN
4,1004.0,B.E.,Computer Science & Engineering,"English, Hindi, Marathi, Marwari",XML; C; Java; Data Structures; Python; MongoDB...,Currently a final year student of Computer Eng...
5,1005.0,B.E.,Computer Science & Engineering,"English,Hindi ,Kashmiri , Urdu",XML; Word; Data Structures; Communication; GUI...,To have a growth oriented and challenging care...


In [21]:
#handling the missing values which were pointed out previously during the analysis
#filling the missing values as Unknown
user_profile_ds['known_languages'] = user_profile_ds['known_languages'].fillna('Unknown')
user_profile_ds['key_skills'] = user_profile_ds['key_skills'].fillna('Unknown')
user_profile_ds['career_objective'] = user_profile_ds['career_objective'].fillna('Unknown')

In [22]:
user_profile_ds['known_languages'] = user_profile_ds['known_languages'].apply(splitting_comma)
user_profile_ds['key_skills'] = user_profile_ds['key_skills'].apply(splitting_semicolon)

In [23]:
#to be consistent with the course dataset by removing the &
user_profile_ds["degree_1_specializations"] = user_profile_ds["degree_1_specializations"].replace("Computer Science & Engineering","Computer Science Engineering")


In [24]:
user_profile_ds

,userid,degree_1,degree_1_specializations,known_languages,key_skills,career_objective
1,1001.0,B.E.,Computer Science Engineering,"[English, Marathi, Hindi]","[C, Java, Keras, Flask, Deep Learning, Se...",Computer Engineering student with good technic...
2,1002.0,B.E.,Computer Science Engineering,[Hindi English],"[Java, Neural Networks, AI, Python, Html5,...",Interested in working under company offering A...
3,1003.0,B.E.,Computer Science Engineering,[Unknown],[Unknown],Unknown
4,1004.0,B.E.,Computer Science Engineering,"[English, Hindi, Marathi, Marwari]","[XML, C, Java, Data Structures, Python, M...",Currently a final year student of Computer Eng...
5,1005.0,B.E.,Computer Science Engineering,"[English, Hindi , Kashmiri , Urdu]","[XML, Word, Data Structures, Communication,...",To have a growth oriented and challenging care...
...,...,...,...,...,...,...
1038,2042.0,B.E.,Computer Science Engineering,"[CPP, Python]","[java, database, html, OOPs, Core Java, M...",i have to be carrer in programming and after t...
1039,2043.0,B.E.,Computer Science Engineering,"[CPP, C]","[Ability-to-cope-up-with-different-situation.,...",To work in an Industry with a professional wor...
1040,2044.0,B.E.,Computer Science Engineering,"[HTML, CSS]","[C, Java, cpp, HTML, Basic-Python, MySQL]",To pursue a highly challenging and creative ca...
1041,2045.0,B.E.,Computer Science Engineering,[Html; Wordpress; Css],"[C, Drupal-(CMS), Bootstrap, Wordpress-(CMS...",To prove myself dedicated worthful and energet...


In [25]:
courses_ds = clean_column_names(courses_ds)
courses_ds.head()

,sr_,degree_1,degree_1_specializations,campus,key_skills
0,1001,B E,Mechanical,MITCOE,CATIA
1,1002,B E,Mechanical,MITCOE,CATIA
2,1003,B E,Mechanical,MITAOE,CATIA
3,1004,B E,Mechanical,MITCOE,CATIA
4,1005,B E,Mechanical,MITCOE,CATIA


In the user profile dataset in Degree 1 BE is written as follows 'B.E' however, in the course dataset it is written as 'B E'. Therefore, to keep consistency the column degree_1 in the course dataset is arranged accordingly to match the column in the user profile. 

In [26]:
# Replace "B E" with "B.E." within each string of the "degree_1" column
courses_ds["degree_1"] = courses_ds["degree_1"].str.replace("B E", "B.E.")

# Display the first few rows to verify the change
courses_ds.head()


,sr_,degree_1,degree_1_specializations,campus,key_skills
0,1001,B.E.,Mechanical,MITCOE,CATIA
1,1002,B.E.,Mechanical,MITCOE,CATIA
2,1003,B.E.,Mechanical,MITAOE,CATIA
3,1004,B.E.,Mechanical,MITCOE,CATIA
4,1005,B.E.,Mechanical,MITCOE,CATIA


In [27]:
courses_ds['key_skills'] = courses_ds['key_skills'].apply(splitting_semicolon)

In [28]:
courses_ds["degree_1_specializations"] = courses_ds["degree_1_specializations"].replace("Computer Science  Engineering","Computer Science Engineering")

In [29]:
courses_ds

,sr_,degree_1,degree_1_specializations,campus,key_skills
0,1001,B.E.,Mechanical,MITCOE,[CATIA]
1,1002,B.E.,Mechanical,MITCOE,[CATIA]
2,1003,B.E.,Mechanical,MITAOE,[CATIA]
3,1004,B.E.,Mechanical,MITCOE,[CATIA]
4,1005,B.E.,Mechanical,MITCOE,[CATIA]
...,...,...,...,...,...
9995,10996,B.E.,Electronics Telecommunication Engineering,MITCOE,"[EmbeddedC, MATLAB, Cprogramming, Keil]"
9996,10997,B.E.,Electronics Telecommunication Engineering,"MIT,Pune","[EmbeddedC, MATLAB, Cprogramming, Keil]"
9997,10998,M TeCh,Electronics Telecommunication Engineering,MIT WPU,"[EmbeddedC, MATLAB, Cprogramming, Keil]"
9998,10999,B.E.,Electronics Telecommunication Engineering,MITAOE,"[AmazonWebServiCes, C CPP, Arduino, MongoDB, L..."


converting skills into a standardized, easily processable format is a preparatory step that supports the implementation of a hybrid recommendation system. It enables the system to leverage both the content attributes of items and users (content-based filtering) and the patterns of user-item interactions (collaborative filtering) to produce more accurate and relevant recommendations.

In [30]:
#For Clustering
courses_ds['key_skills_str'] = courses_ds.key_skills.apply(lambda x: ', '.join([str(i) for i in x]))
user_profile_ds['key_skills_str'] = user_profile_ds.key_skills.apply(lambda x: ', '.join([str(i) for i in x]))

In [31]:
courses_ds

,sr_,degree_1,degree_1_specializations,campus,key_skills,key_skills_str
0,1001,B.E.,Mechanical,MITCOE,[CATIA],CATIA
1,1002,B.E.,Mechanical,MITCOE,[CATIA],CATIA
2,1003,B.E.,Mechanical,MITAOE,[CATIA],CATIA
3,1004,B.E.,Mechanical,MITCOE,[CATIA],CATIA
4,1005,B.E.,Mechanical,MITCOE,[CATIA],CATIA
...,...,...,...,...,...,...
9995,10996,B.E.,Electronics Telecommunication Engineering,MITCOE,"[EmbeddedC, MATLAB, Cprogramming, Keil]","EmbeddedC, MATLAB, Cprogramming, Keil"
9996,10997,B.E.,Electronics Telecommunication Engineering,"MIT,Pune","[EmbeddedC, MATLAB, Cprogramming, Keil]","EmbeddedC, MATLAB, Cprogramming, Keil"
9997,10998,M TeCh,Electronics Telecommunication Engineering,MIT WPU,"[EmbeddedC, MATLAB, Cprogramming, Keil]","EmbeddedC, MATLAB, Cprogramming, Keil"
9998,10999,B.E.,Electronics Telecommunication Engineering,MITAOE,"[AmazonWebServiCes, C CPP, Arduino, MongoDB, L...","AmazonWebServiCes, C CPP, Arduino, MongoDB, Li..."


#### User Rating 


In [32]:
user_rating_cleaned_ds = pd.DataFrame(columns=['course_id', 'user_id', 'rating'])
user_rating_cleaned_ds.head()

,course_id,user_id,rating


In [33]:
# Initialize an empty list to store the rows
cleaned_data = []

#first column of user_rating_ds holds the course_id
course_id_column = user_rating_ds.columns[0]

# Iterate over each row in the original DataFrame
for i, row in user_rating_ds.iterrows():
    # Extract the course_id from the first column
    course_id = row[course_id_column]
    # Iterate over each column in the row, except for the first column which is course_id
    for user in user_rating_ds.columns[1:]:
        # Append a new dictionary to the list for each rating
        cleaned_data.append({
            'course_id': course_id,
            'user_id': user,
            'rating': row[user]
        })

# Once the list is constructed, convert it to a DataFrame
user_rating_cleaned_ds = pd.DataFrame(cleaned_data)

# Now let's ensure that 'rating' column is numeric 
user_rating_cleaned_ds['rating'] = pd.to_numeric(user_rating_cleaned_ds['rating'], errors='coerce')


In [34]:
print(user_rating_cleaned_ds.shape)
user_rating_cleaned_ds


(8480, 3)


,course_id,user_id,rating
0,2001,1001,5
1,2001,1002,3
2,2001,1003,1
3,2001,1004,0
4,2001,1005,2
...,...,...,...
8475,2424,1016,5
8476,2424,1017,4
8477,2424,1018,1
8478,2424,1019,2


In [35]:
#to check how many times a particular rating has been given
user_rating_cleaned_ds["rating"].value_counts()

rating
4     1828
0     1634
3     1527
2     1458
5     1274
1      756
6        1
51       1
11       1
Name: count, dtype: int64

In [36]:
# filter out invalid ratings as from the document it stated that 0 shows a rating has not been assigned whilst 1-5 indicates the rating given. Therefore, 6, 51 and 11 are invalid
user_rating_cleaned_ds = user_rating_cleaned_ds.dropna(subset=['rating'])
user_rating_cleaned_ds = user_rating_cleaned_ds[user_rating_cleaned_ds['rating'] < 6]

In [37]:
user_rating_cleaned_ds["rating"].value_counts()

rating
4    1828
0    1634
3    1527
2    1458
5    1274
1     756
Name: count, dtype: int64

### Saving the cleaned dataframes

In [38]:
user_profile_ds.to_csv('cleanedDatasets/cleaned_user_profile.csv', index=False)
user_rating_cleaned_ds.to_csv('cleanedDatasets/cleaned_user_ratings.csv', index=False)
courses_ds.to_csv('cleanedDatasets/cleaned_course_dataset.csv', index=False)
